In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import timm
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import numpy as np
import random

seed = 2024
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Split the training set into a subset of 40 samples and the rest
subset_size = 400
remaining_size = len(trainset) - subset_size
subset_trainset, remaining_trainset = random_split(trainset, [subset_size, remaining_size])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
trainloader = DataLoader(subset_trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Load the ResNet model from timm
model = timm.create_model('resnet101', pretrained=False, num_classes=10)
model = model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')



In [ ]:
from sklearn.metrics import f1_score
# Test the model
correct = 0
total = 0
y_true = []
y_pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = correct / total
f1 = f1_score(y_true, y_pred, average='macro')

# print('Accuracy: %.4f' % accuracy)
# print('F1: %.4f' % f1)

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import timm
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import numpy as np
import random
from sklearn.metrics import f1_score

seed = 2024
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Split the training set into a subset of 40 samples and the rest
subset_size = 400
remaining_size = len(trainset) - subset_size
subset_trainset, remaining_trainset = random_split(trainset, [subset_size, remaining_size])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
trainloader = DataLoader(subset_trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()

# Train and evaluate models
models = ['resnet18', 'resnet34', 'resnet50', 'resnet101']
for model_name in models:
    print(f'Training {model_name}...')
    model = timm.create_model(model_name, pretrained=False, num_classes=10)
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Train the model
    num_epochs = 50
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 200 == 199:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}')
                running_loss = 0.0

    print('Finished Training')

    # Test the model
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = correct / total
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f'{model_name} Accuracy: {accuracy:.4f}')
    print(f'{model_name} F1: {f1:.4f}')
    print()


100%|██████████| 170498071/170498071 [00:15<00:00, 11218817.58it/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Training resnet18...
Finished Training
resnet18 Accuracy: 0.262
resnet18 F1: 0.2424 

Training resnet34...
Finished Training
resnet34 Accuracy: 0.3796
resnet34 F1: 0.2538

Training resnet50...
Finished Training
resnet50 Accuracy: 0.4336
resnet50 F1: 0.4267

Training resnet101...
Finished Training
resnet101 Accuracy: 0.4462
resnet101 F1: 0.4354 



In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import timm
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import numpy as np
import random
from sklearn.metrics import f1_score

seed = 2024
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the training set into 40 samples per class
subset_indices = []
for class_idx in range(10):
    class_indices = [idx for idx, (_, label) in enumerate(trainset) if label == class_idx]
    subset_indices.extend(random.sample(class_indices, 40))

subset_trainset = Subset(trainset, subset_indices)

# Create data loaders
trainloader = DataLoader(subset_trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()

# Train and evaluate models
models = ['resnet18', 'resnet34', 'resnet50', 'resnet101']
for model_name in models:
    print(f'Training {model_name}...')
    model = timm.create_model(model_name, pretrained=False, num_classes=10)
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Train the model
    num_epochs = 25
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 200 == 199:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}')
                running_loss = 0.0
    print('Finished Training')

    # Test the model
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = correct / total
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f'{model_name} Accuracy: {accuracy:.4f}')
    print(f'{model_name} F1: {f1:.4f}')
    print()


100%|██████████| 170498071/170498071 [00:15<00:00, 11218817.58it/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Training resnet18...
Finished Training
resnet18 Accuracy: 0.2852
resnet18 F1: 0.2674

Training resnet34...
Finished Training
resnet34 Accuracy: 0.3719
resnet34 F1: 0.2838 

Training resnet50...
Finished Training
resnet50 Accuracy: 0.4881
resnet50 F1: 0.4794

Training resnet101...
Finished Training
resnet101 Accuracy: 0.4783
resnet101 F1: 0.4531 

